# 02 - Sentiment Analysis
# RoboReviews | AI Engineer Bootcamp

## 1. Imports & Setup
## 2. Load & Prepare Data
   - Load sentiment_ready.csv
   - Handle class imbalance
   - Train/val/test split
## 3. Baseline — TF-IDF + Logistic Regression
   - Vectorize
   - Train
   - Evaluate (accuracy, F1, confusion matrix)
## 4. Model 2 — Cardiff RoBERTa (zero-shot)
   - Load model
   - Run inference
   - Evaluate same metrics
## 5. Model 3 — Fine-tuned RoBERTa
   - Tokenize
   - Fine-tune
   - Evaluate same metrics
## 6. Comparison & Conclusions
   - Side-by-side metrics table
   - Failure case analysis

In [ ]:
%pip install seaborn

In [ ]:
# =============================================================
# 02 - Sentiment Analysis
# =============================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    ConfusionMatrixDisplay
)
from sklearn.utils.class_weight import compute_class_weight

# Reproducibility
RANDOM_STATE = 42